In [4]:
import json
import w3lib.html
import random
import os
from datetime import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession
from requests_html import HTML
import validators
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

#### Publimetro no redirecciona al momento de buscar noticias de dias anteriores!!!!

In [5]:
URL_SEED = 'https://www.publimetro.cl/noticias/'
XPATH_HREF = "//div[@class='results-list--headline-container']/a/@href"

In [6]:
class Crawler():
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    #Ruta XPATH hacia los enlaces que queremos recopilar
    xpath_href=XPATH_HREF

    #variable que permite listar todos los enlaces recopilado
    url_list = []
    
    def _init_(self, URL_SEED, XPATH_HREF):        
        pass

    
    def get(self,url):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response
    
    def parse(self, response):
        self.url_list = []
        urls = response.html.xpath(self.xpath_href)
        for url in urls:
            self.url_list.append(url)


In [8]:
urls=[]

crawler = Crawler()
response = crawler.get(URL_SEED)
crawler.parse(response)

    
for url in crawler.url_list:
    urls.append("https://www.publimetro.cl"+url)

crawler.url_list = urls
crawler.url_list

['https://www.publimetro.cl/noticias/2022/03/27/madre-de-zamudio-llama-al-gobierno-a-endurecer-la-ley-antidiscriminacion/',
 'https://www.publimetro.cl/noticias/2022/03/27/lo-que-no-sabias-que-necesitabas-se-inaugurara-la-primera-panaderia-popular-en-santiago/',
 'https://www.publimetro.cl/noticias/2022/03/27/en-medio-de-la-guerra-ucrania-anuncia-el-regreso-a-clases-a-distancia/',
 'https://www.publimetro.cl/noticias/2022/03/27/pdi-incauta-yegua-por-presunto-caso-de-maltrato-animal-en-maipu/',
 'https://www.publimetro.cl/noticias/2022/03/27/cancilleria-confirma-la-designacion-de-barbara-figueroa-como-embajadora-de-chile-en-argentina/',
 'https://www.publimetro.cl/noticias/2022/03/27/revelan-video-de-carabinero-atacado-durante-marcha-estudiantil/',
 'https://www.publimetro.cl/noticias/2022/03/27/rinden-homenaje-a-victimas-de-homotransfobia-en-memorial-daniel-zamudio/',
 'https://www.publimetro.cl/noticias/2022/03/27/le-fueron-a-cobrar-hombre-mata-a-su-prestamista-de-un-escopetazo-en-el-

In [9]:
len(urls)

15

In [10]:
urls[0]

'https://www.publimetro.cl/noticias/2022/03/27/madre-de-zamudio-llama-al-gobierno-a-endurecer-la-ley-antidiscriminacion/'

In [11]:
URL_NEWS = urls[0]
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//article/p"

In [12]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = XPATH_TITLE
    query_extract_text =  XPATH_CONTENT

    def _init_(self):
        pass
   
    def get(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers) 
        return response
    
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    
    #### main function ####
    def parse(self, response):
        
        #URL
        self.news_url = response.url
        
        #TITLE
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.news_title = self.clean_text(self.news_title)
       
        
        #CONTENT
        self.news_text = ""
        
        html_paragraphs = response.html.xpath(self.query_extract_text)
        
        for paragraph in html_paragraphs:
            paragraph_content = w3lib.html.remove_tags(paragraph.text)    
            text_cleaned = self.clean_text(paragraph_content)
            self.news_text = self.news_text+text_cleaned


In [13]:
scraper = Scraper()
response =scraper.get(URL_NEWS)

In [14]:
scraper.parse(response)

In [15]:
scraper.news_title

'Madre de Zamudio llama al gobierno a “endurecer la ley antidiscriminación”'

In [16]:
scraper.news_text

'Este domingo 27 de marzo se cumplió una década del brutal crimen en contra de Daniel Zamudio, joven de 24 años que fue torturado por un grupo de personas en el Parque San Borja de Santiago, agresión que terminó por quitarle la vida.Es en el marco de la conmemoración de su muerte que Jaqueline Vera, madre del joven fallecido, se refirió en conversación con Meganoticias a los 10 años del ataque homofóbico en contra de su hijo, y aprovechó de hacer un llamado al gobierno entrante.“Hay que endurecer la ley antidiscriminación. Eso le pido al nuevo presidente. Y también solicitar ayuda para mi fundación, como mamá de la diversidad. No queremos que sigan estos crímenes”, señaló.La mujer también valoró las iniciativas legislativas que condenan este y otros tipos de violencias discriminatorias, como la homónima Ley Zamudio, que ya lleva casi 10 años publicada.“Por lo que le pasó a Daniel, se sacaron varias leyes a favor de la diversidad. A mi hijo solo puedo decirle mil gracias por todo lo que

In [17]:
#Verificamos que el scraper devuelve algo
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)

In [18]:
df = pd.DataFrame(data={'url': [URL_NEWS],'title':[scraper.news_title], 'content':[scraper.news_text]})
df

,url,title,content
0,https://www.publimetro.cl/noticias/2022/03/27/...,Madre de Zamudio llama al gobierno a “endurece...,Este domingo 27 de marzo se cumplió una década...


In [19]:
XPATH_TITLE = "//h1"
XPATH_CONTENT ="//div[@class='single-content']//p"

for url in tqdm(urls):
    URL_NEWS = url
    scraper = Scraper()
    response =scraper.get(URL_NEWS)
    scraper.parse(response)
    df1 = pd.DataFrame(data={'url': url, 'title':scraper.news_title, 'content':scraper.news_text},index=['url', 'title', 'content'])
    df = df.append(df1, ignore_index=True)
df = df.drop_duplicates()


  0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
df

,url,title,content
0,https://www.publimetro.cl/noticias/2022/03/27/...,Madre de Zamudio llama al gobierno a “endurece...,Este domingo 27 de marzo se cumplió una década...
4,https://www.publimetro.cl/noticias/2022/03/27/...,Lo que no sabías que necesitabas: Se inaugurar...,La Municipalidad de Pedro Aguirre Cerda anunci...
7,https://www.publimetro.cl/noticias/2022/03/27/...,"En medio de la guerra, Ucrania anuncia el regr...","El alcalde de Kiev, Vitali Klitschko, ha anunc..."
10,https://www.publimetro.cl/noticias/2022/03/27/...,PDI incauta yegua por presunto caso de maltrat...,Detectives de la Brigada Investigadora de Deli...
13,https://www.publimetro.cl/noticias/2022/03/27/...,Cancillería confirma la designación de Bárbara...,Durante la tarde de este domingo y mediante un...
16,https://www.publimetro.cl/noticias/2022/03/27/...,Revelan video de carabinero atacado durante ma...,El diputado Jorge Alessandri (UDI) publicó un ...
19,https://www.publimetro.cl/noticias/2022/03/27/...,Rinden homenaje a víctimas de homo/transfobia ...,"A diez años del asesinato de Daniel Zamudio, e..."
22,https://www.publimetro.cl/noticias/2022/03/27/...,Le fueron a cobrar: hombre mata a su prestamis...,"La noche de este sábado, efectivos policiales ..."
25,https://www.publimetro.cl/noticias/2022/03/27/...,Tragedia en Colchane: Bebé de 6 meses muere cr...,Dos nuevas víctimas fatales fueron constatadas...
28,https://www.publimetro.cl/noticias/2022/03/27/...,En riesgo vital carabinero atropellado por mot...,En riesgo vital se encuentra el Cabo 2° Álvaro...
